In [ ]:
import pandas as pd
import geopandas as gpd
import maup
import numpy as np
import geopandas as gpd
from shapely.ops import nearest_points
import os

# Current Working Directory
working_directory = "/Users/stanleymui/Downloads/CSE 416 Preprocessing Data/"

In [ ]:
def find_neighbor_adjacency(working_directory, file, state):
    
    gdf = gpd.read_file(working_directory + file)
    
    boundary_length_threshold = 200
    edge_distance_threshold = 200    
    
    boundary_length_threshold_m = boundary_length_threshold * 0.3048
    edge_distance_threshold_m = edge_distance_threshold * 0.3048
    
    gdf = gdf.to_crs("EPSG:32610")
    
    gdf["NEIGHBORS"] = None  
    
    for index, precinct in gdf.iterrows():
        potential_neighbors = gdf[~gdf.geometry.disjoint(precinct.geometry)]
        
        neighbors = []
        
        for _, neighbor in potential_neighbors.iterrows():
            
            if precinct.SRPREC_KEY == neighbor.SRPREC_KEY:
                continue
            
            shared_boundary = precinct.geometry.intersection(neighbor.geometry).length
            
            if shared_boundary >= boundary_length_threshold_m:
                p1, p2 = nearest_points(precinct.geometry.boundary, neighbor.geometry.boundary)
                edge_distance = p1.distance(p2)
                
                if edge_distance <= edge_distance_threshold_m:
                    neighbors.append(neighbor.SRPREC_KEY)
        
        gdf.at[index, "NEIGHBORS"] = ", ".join(neighbors)
    
    display(gdf[["SRPREC_KEY", "NEIGHBORS"]])

    gdf = gdf[["SRPREC_KEY", "NEIGHBORS"]]
    
    file_path = os.path.join(working_directory, state + "_neighbors.json")

    json_data = gdf.to_json()
    
    with open(file_path, "w") as file:
        file.write(json_data)

if __name__ == "__main__":
    file = "california_precinct_merged.geojson"
    find_neighbor_adjacency(working_directory, file, 'california')
    file = "alabama_precinct_merged.geojson"
    find_neighbor_adjacency(working_directory, file, 'alabama')
